# Calculation of Bayesian Cramér-Rao bounds

This is the source codes for the example discussed in Sec. III C and D in Ref. [1].

The Hamiltonian of a qubit system under a magnetic field $B$ in the XZ plane is
\begin{align}
H=\frac{B\omega_0}{2}(\sigma_1\cos{x}+\sigma_3\sin{x})
\end{align}

with $x$ the unknown parameter and $\sigma_{1}$, $\sigma_{3}$ Pauli matrices.

The probe state is taken as $\frac{1}{\sqrt{2}}(|0\rangle+|1\rangle)$ with $|0\rangle$ and $|1\rangle$ the eigenvstates of $\sigma_3$ with respect to the eigenvalues $1$ and $-1$. The measurement for classical bounds is a set of rank-one symmetric informationally complete positive operator-valued measure (SIC-POVM).

Take the Gaussian prior distribution $p(x)=\frac{1}{c\eta\sqrt{2\pi}}\exp\left({-\frac{(x-\mu)^2}{2\eta^2}}\right)$ on $[-\pi/2, \pi/2]$
with $\mu$ and $\eta$ the expectation and standard deviation, respectively. 
Here $c=\frac{1}{2}\big[\mathrm{erf}(\frac{\pi-2\mu}{2\sqrt{2}\eta})+\mathrm{erf}(\frac{\pi+2\mu}{2\sqrt{2}\eta})\big]$ 
is the normalized coefficient with $\mathrm{erf}(x):=\frac{2}{\sqrt{\pi}}\int^x_0 e^{-t^2}\mathrm{d}t$ the error 
function.

[1] M. Zhang et al., QuanEstimation: an open-source toolkit for quantum parameter estimation,
arXiv:2022.xxxx.

In [5]:
from quanestimation import *
import numpy as np
from scipy.integrate import simps

# initial state
rho0 = 0.5*np.array([[1., 1.], [1., 1.]])
# free Hamiltonian
B, omega0 = 0.5*np.pi, 1.0
sx = np.array([[0., 1.], [1., 0.]])
sy = np.array([[0., -1.j], [1.j, 0.]]) 
sz = np.array([[1., 0.], [0., -1.]])
H0_func = lambda x: 0.5*B*omega0*(sx*np.cos(x)+sz*np.sin(x))
# derivative of the free Hamiltonian on x
dH_func = lambda x: [0.5*B*omega0*(-sx*np.sin(x)+sz*np.cos(x))]
# prior distribution
x = np.linspace(-0.5*np.pi, 0.5*np.pi, 100)
mu, eta = 0.0, 0.2
p_func = lambda x, mu, eta: np.exp(-(x-mu)**2/(2*eta**2)) \
                            /(eta*np.sqrt(2*np.pi))
dp_func = lambda x, mu, eta: -(x-mu)*np.exp(-(x-mu)**2/(2*eta**2)) \
                              /(eta**3*np.sqrt(2*np.pi))
p_tp = [p_func(x[i], mu, eta) for i in range(len(x))]
dp_tp = [dp_func(x[i], mu, eta) for i in range(len(x))]
# normalization of the distribution
c = simps(p_tp, x)
p, dp = p_tp/c, dp_tp/c
# time length for the evolution
tspan = np.linspace(0., 1., 1000)
# dynamics
rho = [np.zeros((len(rho0), len(rho0)), dtype=np.complex128) \
       for i in range(len(x))]
drho = [[np.zeros((len(rho0), len(rho0)), dtype=np.complex128)] \
         for i in range(len(x))]
for i in range(len(x)):
    H0_tp = H0_func(x[i])
    dH_tp = dH_func(x[i])
    dynamics = Lindblad(tspan, rho0, H0_tp, dH_tp)
    rho_tp, drho_tp = dynamics.expm()
    rho[i] = rho_tp[-1]
    drho[i] = drho_tp[-1]

Classical Bayesian bounds

In [6]:
f_BCRB1 = BCRB([x], p, [], rho, drho, M=[], btype=1)
f_BCRB2 = BCRB([x], p, [], rho, drho, M=[], btype=2)
f_BCRB3 = BCRB([x], p, dp, rho, drho, M=[], btype=3)
f_VTB = VTB([x], p, dp, rho, drho, M=[])

Quantum Bayesian bounds

In [7]:
f_BQCRB1 = BQCRB([x], p, [], rho, drho, btype=1)
f_BQCRB2 = BQCRB([x], p, [], rho, drho, btype=2)
f_BQCRB3 = BQCRB([x], p, dp, rho, drho, btype=3)
f_QVTB = QVTB([x], p, dp, rho, drho)
f_QZZB = QZZB([x], p, rho)